# Air Quality Analysis Agent using PraisonAIAgents

## Description

This notebook demonstrates how to build an Air Quality Analysis Agent using the `praisonaiagents` library.  
The agent retrieves real-time air quality data for a specified location, analyzes the results, and provides actionable insights or health recommendations.  
You will learn how to set up the environment, configure API keys, use custom tools, and orchestrate the agent workflow with a YAML prompt.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/examples/cookbooks/Health_Fitness_Agents/Air_Quality_Analysis_Agent.ipynb)


# Dependencies

In [ ]:
!pip install praisonaiagents firecrawl requests

# Set up API Key

In [32]:
# Replace with your actual API keys
FIRECRAWL_API_KEY = "fc-..."  # Your Firecrawl API key
OPENAI_API_KEY = "sk-..."     # Your OpenAI API key

import os
os.environ["FIRECRAWL_API_KEY"] = FIRECRAWL_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# Tools

In [33]:
import requests
from firecrawl import FirecrawlApp
from typing import Dict

class AQIDataFetcher:
    def __init__(self, firecrawl_key: str):
        self.firecrawl = FirecrawlApp(api_key=firecrawl_key)

    def fetch(self, country: str, state: str, city: str) -> Dict:
        country_clean = country.lower().replace(' ', '-')
        city_clean = city.lower().replace(' ', '-')
        if not state or state.lower() == 'none':
            url = f"https://www.aqi.in/dashboard/{country_clean}/{city_clean}"
        else:
            state_clean = state.lower().replace(' ', '-')
            url = f"https://www.aqi.in/dashboard/{country_clean}/{state_clean}/{city_clean}"

        # Check if the URL is valid and reachable
        print(f"Testing URL: {url}")
        try:
            resp = requests.get(url)
            print(f"Status code: {resp.status_code}")
            if resp.status_code != 200:
                print(f"URL not reachable: {url} (status code: {resp.status_code})")
                return None
        except Exception as e:
            print(f"Error: {e}")
            return None

        schema = {
            "type": "object",
            "properties": {
                "aqi": {"type": "number", "description": "Air Quality Index"},
                "temperature": {"type": "number", "description": "Temperature in degrees Celsius"},
                "humidity": {"type": "number", "description": "Humidity percentage"},
                "wind_speed": {"type": "number", "description": "Wind speed in kilometers per hour"},
                "pm25": {"type": "number", "description": "Particulate Matter 2.5 micrometers"},
                "pm10": {"type": "number", "description": "Particulate Matter 10 micrometers"},
                "co": {"type": "number", "description": "Carbon Monoxide level"}
            },
            "required": ["aqi", "temperature", "humidity", "wind_speed", "pm25", "pm10", "co"]
        }

        response = self.firecrawl.extract(
            urls=[f"{url}/*"],
            prompt='Extract the current real-time AQI, temperature, humidity, wind speed, PM2.5, PM10, and CO levels from the page. Also extract the timestamp of the data.',
            schema=schema
        )
        return response

# YAML Prompt

In [34]:
import yaml

yaml_prompt = """
agent:
  name: Air Quality Health Advisor
  description: |
    An agent that analyzes real-time air quality data for a given location and provides personalized health recommendations based on user context.
  tools:
    - name: AQIDataFetcher
      description: Fetches AQI and weather data for a specified city, state, and country using Firecrawl.
  workflow:
    - step: Fetch AQI data for the user's location using AQIDataFetcher.
    - step: Analyze AQI and weather data.
    - step: Consider user's medical conditions and planned activity.
    - step: Generate health and activity recommendations.
inputs:
  - city
  - state
  - country
  - medical_conditions
  - planned_activity
outputs:
  - recommendations
"""

print(yaml.safe_load(yaml_prompt))

{'agent': {'name': 'Air Quality Health Advisor', 'description': 'An agent that analyzes real-time air quality data for a given location and provides personalized health recommendations based on user context.\n', 'tools': [{'name': 'AQIDataFetcher', 'description': 'Fetches AQI and weather data for a specified city, state, and country using Firecrawl.'}], 'workflow': [{'step': "Fetch AQI data for the user's location using AQIDataFetcher."}, {'step': 'Analyze AQI and weather data.'}, {'step': "Consider user's medical conditions and planned activity."}, {'step': 'Generate health and activity recommendations.'}]}, 'inputs': ['city', 'state', 'country', 'medical_conditions', 'planned_activity'], 'outputs': ['recommendations']}


# Main

In [40]:
from praisonaiagents import Agent
import os

# User input (you can change these for other locations)
user_city = "Mumbai"
user_state = "Maharashtra"
user_country = "India"
user_medical_conditions = "asthma"
user_planned_activity = "morning jog for 2 hours"

aqi_fetcher = AQIDataFetcher(firecrawl_key=os.environ["FIRECRAWL_API_KEY"])

# Try with state, then without state, then fallback to Delhi
aqi_data = aqi_fetcher.fetch(
    country=user_country,
    state=user_state,
    city=user_city
)

if not aqi_data or not hasattr(aqi_data, "data"):
    print("Mumbai with state failed, trying without state...")
    aqi_data = aqi_fetcher.fetch(
        country=user_country,
        state="",
        city=user_city
    )

if not aqi_data or not hasattr(aqi_data, "data"):
    print("Trying Delhi, India...")
    aqi_data = aqi_fetcher.fetch(
        country="India",
        state="",
        city="Delhi"
    )

if not aqi_data or not hasattr(aqi_data, "data"):
    print("Failed to fetch AQI data for all tried locations. Please check the URLs manually.")
else:
    aqi_data = aqi_data.data  # Convert to dict

    prompt = f"""
    You are an expert health advisor using the gpt-5-nano Mini model.

    Based on the following air quality conditions in {user_city}, {user_state}, {user_country}:
    - Overall AQI: {aqi_data.get('aqi', 'N/A')}
    - PM2.5 Level: {aqi_data.get('pm25', 'N/A')} µg/m³
    - PM10 Level: {aqi_data.get('pm10', 'N/A')} µg/m³
    - CO Level: {aqi_data.get('co', 'N/A')} ppb

    Weather conditions:
    - Temperature: {aqi_data.get('temperature', 'N/A')}°C
    - Humidity: {aqi_data.get('humidity', 'N/A')}%
    - Wind Speed: {aqi_data.get('wind_speed', 'N/A')} km/h

    User's Context:
    - Medical Conditions: {user_medical_conditions or 'None'}
    - Planned Activity: {user_planned_activity}

    **Comprehensive Health Recommendations:**
    1. **Impact of Current Air Quality on Health:**
    2. **Necessary Safety Precautions for Planned Activity:**
    3. **Advisability of Planned Activity:**
    4. **Best Time to Conduct the Activity:**
    """

    agent = Agent(
        name="Air Quality Health Advisor",
        instructions=prompt,
        api_key=os.environ["OPENAI_API_KEY"]
    )

    # Call run() with no arguments
recommendations = agent.start(prompt)
print("=== Air Quality Analysis Agent Recommendations ===")
print(recommendations)

Testing URL: https://www.aqi.in/dashboard/india/maharashtra/mumbai
Status code: 200


╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Air Quality Health Advisor                                                                           │
│  Role: Assistant                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Response generated in 10.8s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ You are an expert health advisor using the gpt-5-nano Mini model.                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  Based on the following air quality conditions in Mumbai, Maharashtra, India:                                   │
│  - Overall AQI: 75                                                                                              │
│  - PM2.5 Level: 22 µg/m³                                                                                        │
│  - PM10 Level: 60 µg/m³                                                                                         │
│  - CO Level: 650 ppb                                                                                            │
│                                                                                                                 │
│  Weather conditions:                                                                                            │
│  - Temperature: 28°C                                                                                            │
│  - Humidity: 84%                                                                                                │
│  - Wind Speed: 30 km/h                                                                                          │
│                                                                                                                 │
│  User's Context:                                                                                                │
│  - Medical Conditions: asthma                                                                                   │
│  - Planned Activity: morning jog for 2 hours                                                                    │
│                                                                                                                 │
│  **Comprehensive Health Recommendations:**                                                                      │
│  1. **Impact of Current Air Quality on Health:**                                                                │
│  2. **Necessary Safety Precautions for Planned Activity:**                                                      │
│  3. **Advisability of Planned Activity:**                                                                       │
│  4. **Best Time to Conduct the Activity:**                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Certainly! Here are the comprehensive health recommendations based on the provided conditions:                  │
│                                                                                                                 │
│  1 Impact of Current Air Quality on Health:                                                                     │
│     • The overall AQI of 75 is considered "Moderate." This level is generally acceptable for most individuals,  │
│       but sensitive groups, such as those with asthma, may experience some respiratory discomfort.              │
│     • PM2.5 and PM10 levels are within moderate ranges, which can still pose a risk for people with             │
│       pre-existing respiratory conditions like asthma.                                                          │
│     • The CO level of 650 ppb is not typically concerning, but it is important to be cautious as individuals    │
│       with asthma may be more sensitive to pollutants.                                                          │
│  2 Necessary Safety Precautions for Planned Activity:                                                           │
│     • Consider wearing a mask designed to filter out particulate matter to reduce inhalation of pollutants.     │
│     • Carry a rescue inhaler and any other prescribed asthma medication.                                        │
│     • Stay hydrated, as high humidity can exacerbate asthma symptoms.                                           │
│     • Monitor your breathing and be prepared to stop the activity if you experience any respiratory distress.   │
│     • Choose a route with less traffic to minimize exposure to vehicle emissions.                               │
│  3 Advisability of Planned Activity:                                                                            │
│     • Given the moderate AQI and the user's asthma condition, it is advisable to proceed with caution. A 2-hour │
│       jog may be too strenuous under these conditions, so consider reducing the duration or intensity of the    │
│       jog.                                                                                                      │
│     • It may be beneficial to consult with a healthcare provider to tailor the activity to your specific health │
│       needs.                                                                                                    │
│  4 Best Time to Conduct the Activity:                                                                           │
│     • Early morning hours, just after sunrise, are typically better for outdoor activities as pollution levels  │
│       tend to be lower.                                                                                         │
│     • Avoid peak traffic times to reduce exposure to vehicle emissions.                                         │
│     • Check real-time air quality updates to ensure conditions remain stable or improve before starting the     │
│       activity.                                                                                                 │
│                                                                                                                 │
│ By following these recommendations, you can help mitigate potential health risks while enjoying your planned    │
│ activity.                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

=== Air Quality Analysis Agent Recommendations ===
Certainly! Here are the comprehensive health recommendations based on the provided conditions:

1. **Impact of Current Air Quality on Health:**
   - The overall AQI of 75 is considered "Moderate." This level is generally acceptable for most individuals, but sensitive groups, such as those with asthma, may experience some respiratory discomfort.
   - PM2.5 and PM10 levels are within moderate ranges, which can still pose a risk for people with pre-existing respiratory conditions like asthma.
   - The CO level of 650 ppb is not typically concerning, but it is important to be cautious as individuals with asthma may be more sensitive to pollutants.

2. **Necessary Safety Precautions for Planned Activity:**
   - Consider wearing a mask designed to filter out particulate matter to reduce inhalation of pollutants.
   - Carry a rescue inhaler and any other prescribed asthma medication.
   - Stay hydrated, as high humidity can exacerbate asthma 